In [1]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import shape_detection
import bounding_box
import copy

images = [basic_ops.load_image(image_path='traffic_sign_images/vfs/vfs_04.jpg'), basic_ops.load_image(image_path='traffic_sign_images/vfs/vfs_05.jpg')]
images = [geo_ops.resize_image(image=image, target_height=image.shape[0]//8, target_width=image.shape[1]//8) for image in images]
orig_images = copy.deepcopy(images)

Image loaded from traffic_sign_images/vfs/vfs_04.jpg
Image loaded from traffic_sign_images/vfs/vfs_05.jpg


In [2]:
for i, image in enumerate(images):
    image = filters.median_filter(image=image, dim=3)
    image = filters.gray_scale_filter(image=image)
    image = filters.sobel_filter(image=image, mode='both')
    image = filters.black_white_filter(image=image, threshold=40)
    images[i] = image

In [20]:
import importlib
importlib.reload(shape_detection)

shape_bounding_boxes = []
shape_contours = []
for i, image in enumerate(images):
    contours = shape_detection.get_contours(binary_image=image, angle_tolerance=10)
    height, width = image.shape[:2]
    box_color = [255, 255, 255]
    min_box_area = int((height*0.055)**2)
    max_box_area = height*width
    shape_contours.extend(contours)
    bounding_boxes = bounding_box.create_bounding_boxes(blobs=shape_contours, min_box_area=min_box_area, max_box_area=max_box_area, box_color=box_color, image_index=i)
    shape_bounding_boxes.extend(bounding_boxes)

In [21]:
import copy
bbox_images = copy.deepcopy(orig_images)
for bounding_box_obj in shape_bounding_boxes:
    print(bounding_box_obj)
    bbox_image = bounding_box.draw_bounding_box(bounding_box=bounding_box_obj, image=bbox_images[bounding_box_obj.image_index])
    bbox_images[bounding_box_obj.image_index] = bbox_image

BoundingBox(center_y=545, center_x=152, box_corners=[np.int64(524), np.int64(130), np.int64(566), np.int64(174)], height=43, width=45, area=1935, box_color=[255, 255, 255], image_index=0)
BoundingBox(center_y=545, center_x=152, box_corners=[np.int64(524), np.int64(130), np.int64(566), np.int64(174)], height=43, width=45, area=1935, box_color=[255, 255, 255], image_index=1)
BoundingBox(center_y=49, center_x=180, box_corners=[np.int64(23), np.int64(149), np.int64(76), np.int64(212)], height=54, width=64, area=3456, box_color=[255, 255, 255], image_index=1)
BoundingBox(center_y=82, center_x=91, box_corners=[np.int64(61), np.int64(70), np.int64(104), np.int64(113)], height=44, width=44, area=1936, box_color=[255, 255, 255], image_index=1)


In [22]:
for bbox_img in bbox_images:
    basic_ops.show_image(image=bbox_img, title='bbox_image')

Image displayed
Image displayed
